In [14]:
import os

os.environ["SYNTHESIA_CLOUD_ENVIRONMENT"] = "aws"
os.environ["SYNTHESIA_WD"] = "/scratch/tmp_kaiwenguo"

from datalib import InputPath
import polars as pl


snapshot_file = InputPath(
    "s3://synthesia-rnd-dataops-prd-datalake/dataset-snapshots/data/snapshot_2e2aa75c_af3f_4820_b9bb_c9d437f038b9"
).resolve()

snapshot_df = pl.scan_parquet(snapshot_file / "**/*.parquet")
snapshot_df = snapshot_df.select(pl.col("video_s3_path")).head(1)
snapshot_df = snapshot_df.collect()
print(snapshot_df["video_s3_path"][0])

s3://synthesia-rnd-dataops-prd-datalake/processed/youtube_human_activity_rml/synthesia-rnd-videotranscoder/transcoded/created_at=20241125/synthesia_id-e0ceafda-ca8d-db35-10c1-d0f99b91f3d5.webm


In [26]:
import sys

sys.path.append("/home/kaiwenguo/dev/rnd-ditwo-reference-cond/src")

from datalib import InputPath

from ditwo.configs.config_dataclasses import DiTTrainingConfig, InferenceConfig
from ditwo.eval.inference import get_dataset

training_config = DiTTrainingConfig()
inference_config = InferenceConfig()
dataset_config = training_config.dataset_config
video_path = InputPath(
    "s3://synthesia-rnd-eun1-dataops-prd-datalake/processed/youtube_human_activity_rml/synthesia-rnd-videotranscoder/transcoded/created_at=20241125/synthesia_id-e0ceafda-ca8d-db35-10c1-d0f99b91f3d5.webm"
).resolve()

frames_dataset = get_dataset(
    dataset_config=dataset_config.model_copy(
        update={
            "n_frames": 1,
            "n_reference_frames": 3,
            "dataset_layers": [
                "random-reference-frames",
                "resize-frames",
                "convert-to-rgb-tensor",
                "center-crop-frames",
                "retry",
            ],
        }
    ),
    video_path=video_path,
    load_frames=True,
    seed=inference_config.seed,
    remove_layers=[
        "skeleton-data",
        "cpu-rendered-dwpose",
    ],
)

LOG: WARNING Frame has unexpected PTS value: 652920, expected: 3600000000


In [28]:
all_clip_ids = list(frames_dataset.get_frames_dataset().metadata_per_id.keys())
clip_id = all_clip_ids[0]

In [40]:
batch = frames_dataset.get_test_item(clip_id, frames_index=[0])
print(batch.reference_frames.shape)

torch.Size([3, 3, 480, 832])
